First step is to import the compiled drill data from Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Gleason Crenshaw Drillhole Compilation').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

Convert row 0 to header

In [ ]:
header_row = 0
df.columns = df.iloc[header_row]
df.head()

In [ ]:
df = df.drop(header_row)
df.head()

In [ ]:
df = df.reset_index(drop=True)
df.head()

Clean the DataFrame by dropping some of the columns that won't be used (the Check From-To and CheckOnelevelUp)

In [ ]:
df = df.drop(columns=['Check From-To', 'CheckOneLevelUp'])

In [ ]:
df.head()

In [ ]:
import numpy as np

Next step is to ensure that the columns to do the summary stats are numeric types

In [ ]:
df.dtypes

Since many of the columns are not numeric, these need to be converted to numeric with the following code:

In [ ]:
df['Total'] = pd.to_numeric(df['Total'], errors='coerce')

In [ ]:
df['sio2_perc'] = pd.to_numeric(df['sio2_perc'], errors='coerce')
df['al2o3_perc'] = pd.to_numeric(df['al2o3_perc'], errors='coerce')
df['fe2o3_perc'] = pd.to_numeric(df['fe2o3_perc'], errors='coerce')
df['tio2_perc'] = pd.to_numeric(df['tio2_perc'], errors='coerce')
df['k2o_perc'] = pd.to_numeric(df['k2o_perc'], errors='coerce')
df['loi'] = pd.to_numeric(df['loi'], errors='coerce')
df['mbi'] = pd.to_numeric(df['mbi'], errors='coerce')
df['carbon'] = pd.to_numeric(df['carbon'], errors='coerce')
df['ssa'] = pd.to_numeric(df['ssa'], errors='coerce')
df['sulfur'] = pd.to_numeric(df['sulfur'], errors='coerce')

In [ ]:
df['Thickness_ft'] = pd.to_numeric(df['Thickness_ft'], errors='coerce')

Check that conversion is correct

In [ ]:
df.dtypes

Create a copy of ofiginal database to obtain a summary of statistics for all the test where value is greater than 0.

In [ ]:
df_clean = df.copy()

In [ ]:
real_values = df_clean._get_numeric_data()
real_values[real_values < 0] = 0
df_clean.mask(df_clean == 0).describe()


There are some testing where clearly the maximum value is not right (SiO2, TiO2). Let find them.

In [ ]:
df_clean[df_clean.sio2_perc > 100]

In [ ]:
df_clean[df_clean.tio2_perc > 100]

In order to obtain a clean summary of all stats, these suspicios values for SiO2 and TiO2 would be replaced for 0s

In [ ]:
df_clean.loc[df_clean['sio2_perc'] > 100, 'sio2_perc'] = 0

In [ ]:
df_clean.loc[df_clean['tio2_perc'] > 100, 'tio2_perc'] = 0

Jackson and Lovelace have the exact same data as both are part of the same group of properties, in order to avoid any bias rows for Lovelace would be dropped.

In [ ]:
df_clean.drop(df_clean[df_clean['Property'] == 'Lovelace'].index, inplace= True)

In [ ]:
df_clean.mask(df_clean == 0).describe()

Summary for each column based on different properties

In [ ]:
df_clean.mask(df_clean == 0).groupby('Property').describe()

Now the descriptive stats per test type and per property would be exported to GS

In [ ]:
summary_by_test = df_clean.mask(df_clean == 0).describe()

Reset index in order to include the column describing the parameters, otherwise it wont't be written in GS. Same for the summary by property.

In [ ]:
summary_by_test = summary_by_test.reset_index()

In [ ]:
from gspread_dataframe import set_with_dataframe

In [ ]:
gc2 = gspread.authorize(GoogleCredentials.get_application_default())
summary = gc2.open('KT EDA Summary tables').get_worksheet(0)

In [ ]:
set_with_dataframe(summary, summary_by_test)

In [ ]:
summary_by_prop = df_clean.mask(df_clean == 0).groupby('Property').describe()

In [ ]:
summary_by_prop = summary_by_prop.reset_index()

In [ ]:
gc3 = gspread.authorize(GoogleCredentials.get_application_default())
summary_prop = gc3.open('KT EDA Summary tables').get_worksheet(1)

In [ ]:
set_with_dataframe(summary_prop, summary_by_prop)

Next we would work on the graphs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [ ]:
df_clean['SampleCount'] = pd.to_numeric(df_clean['SampleCount'], errors='coerce')

In [ ]:
df_clean.head()

In [ ]:
import scipy.stats
from scipy.stats import norm

In [ ]:
multi_graph = sns.FacetGrid(df_clean, col='Property', col_wrap=3, xlim=(40,100), ylim=(0,800))
multi_graph.map(sns.histplot, "sio2_perc")

In [ ]:
multi_graph = sns.FacetGrid(df_clean, col='Property', col_wrap=3, xlim=(0,10))
multi_graph.map(sns.histplot, "Total")

In [ ]:
alumina = df_clean[['Property', 'al2o3_perc']].copy()

In [ ]:
multi_graph = sns.FacetGrid(alumina, col='Property', col_wrap=3, xlim=(0,70), ylim=(0,750))
multi_graph.map(sns.histplot, "al2o3_perc")

In [ ]:
alumina.drop(alumina[alumina['al2o3_perc'] == 0].index, inplace= True)

In [ ]:
silica = df_clean[['Property', 'sio2_perc']].copy()
silica.drop(silica[silica['sio2_perc'] == 0].index, inplace= True)

In [ ]:
iron = df_clean[['Property', 'fe2o3_perc']].copy()
iron.drop(iron[iron['fe2o3_perc'] == 0].index, inplace= True)

In [ ]:
multi_graph = sns.FacetGrid(iron, col='Property', col_wrap=3, xlim=(0,5), ylim=(0,650))
multi_graph.map(sns.histplot, "fe2o3_perc")

In [ ]:
titanium = df_clean[['Property', 'tio2_perc']].copy()
titanium.drop(titanium[titanium['tio2_perc'] == 0].index, inplace= True)

In [ ]:
multi_graph = sns.FacetGrid(titanium, col='Property', col_wrap=3, xlim=(0,4), ylim=(0,800))
multi_graph.map(sns.histplot, "tio2_perc")

In [ ]:
k2o = df_clean[['Property', 'k2o_perc']].copy()
k2o.drop(k2o[k2o['k2o_perc'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(k2o, col='Property', col_wrap=3, xlim=(0,5))
multi_graph.map(sns.histplot, "k2o_perc", binwidth=0.1)

In [ ]:
loi = df_clean[['Property', 'loi']].copy()
loi.drop(loi[loi['loi'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(loi, col='Property', col_wrap=3, xlim=(0,20))
multi_graph.map(sns.histplot, "loi", binwidth=0.5)

In [ ]:
mbi = df_clean[['Property', 'mbi']].copy()
mbi.drop(mbi[mbi['mbi'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(mbi, col='Property', col_wrap=3, xlim=(0,30))
multi_graph.map(sns.histplot, "mbi", binwidth=1)

In [ ]:
carbon = df_clean[['Property', 'carbon']].copy()
carbon.drop(carbon[carbon['carbon'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(carbon, col='Property', col_wrap=3, xlim=(0,1))
multi_graph.map(sns.histplot, "carbon", binwidth=0.05)

In [ ]:
ssa = df_clean[['Property', 'ssa']].copy()
ssa.drop(ssa[ssa['ssa'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(ssa, col='Property', col_wrap=3, xlim=(0,70))
multi_graph.map(sns.histplot, "ssa")

In [ ]:
sulfur = df_clean[['Property', 'sulfur']].copy()
sulfur.drop(sulfur[sulfur['sulfur'] == 0].index, inplace= True)
multi_graph = sns.FacetGrid(sulfur, col='Property', col_wrap=3, xlim=(0,1000))
multi_graph.map(sns.histplot, "sulfur")

In [ ]:
total = df_clean[['Property', 'Total']].copy()
total.drop(total[total['Total'] == 0].index, inplace= True)

In [ ]:
sns.histplot(data=total, x='Total', bins=100)
plt.xlim([0,30])


In [ ]:
sns.histplot(data=silica, x='sio2_perc', binwidth=4)
plt.xlim([40,90])

In [ ]:
sns.histplot(data=alumina, x='al2o3_perc', binwidth=2)
plt.xlim([10,40])

In [ ]:
sns.histplot(data=iron, x='fe2o3_perc', binwidth=0.25)
plt.xlim([0,5])

In [ ]:
sns.histplot(data=titanium, x='tio2_perc', binwidth=0.25)
plt.xlim([0,5])

In [ ]:
sns.histplot(data=k2o, x='k2o_perc', binwidth=0.15)
plt.xlim([0,3])

In [ ]:
sns.histplot(data=loi, x='loi', binwidth=1)
plt.xlim([0,30])

In [ ]:
sns.histplot(data=mbi, x='mbi', binwidth=2)
plt.xlim([0,30])

In [ ]:
sns.histplot(data=carbon, x='carbon', binwidth=0.1)
plt.xlim([0,2])

In [ ]:
sns.histplot(data=ssa, x='ssa', binwidth=2)
plt.xlim([0,50])

In [ ]:
sns.histplot(data=sulfur, x='sulfur', binwidth=10)
plt.xlim([0,1000])

Now let's build the scatterplots to investigate any correlation. The thickness and sample count columns would be removed.

In [ ]:
df_clean2 = df_clean.drop(columns=['Thickness_ft','SampleCount'])

In [ ]:
df_clean2 = df_clean2.drop(columns=['ModifiedHoleID', 'SampleID', 'HoleID', 'From', 'To'])

In [ ]:
df_clean2.head()

In [ ]:
cols = ['sio2_perc', 'al2o3_perc', 'fe2o3_perc', 'tio2_perc', 'k2o_perc', 'loi', 'mbi', 'carbon', 'ssa', 'sulfur']
df_clean2[cols] = df_clean2[cols].replace({0:np.nan, 0:np.nan})

In [ ]:
df_clean2.head()

In [ ]:
corrs = sns.PairGrid(df_clean2, diag_sharey=False, corner=True, dropna=True)
corrs.map_diag(sns.histplot)
corrs.map_lower(sns.scatterplot)

In [ ]:
corrs = sns.PairGrid(df_clean2, diag_sharey=False, corner=True, dropna=True)
corrs.map_diag(sns.histplot)
corrs.map_lower(sns.regplot)

In [ ]:
df_clean2.corr()